The Lagrangian Averaged Vorticity Deviation (LAVD) is computed from the vorticity $ \omega $ as:

\begin{equation}
\mathrm{LAVD}_{t_0}^{t_N} = \dfrac{1}{t_N-t_0} \sum_{i = 0}^{N}| \omega(\mathbf{x}(t_i)) - \overline{\omega}(t_i) |
\end{equation}

where $ \omega(\mathbf{x}(t_i)) $ indicates the vorticity at $ \mathbf{x}(t_i) $. $ \overline{\omega}(t_i) $ is the spatial average of the vorticity at time $ t_i $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| trajectory_grid | array (Ny, Nx, 2, N) | trajectories launched from <br /> meshgrid of size (Ny, Nx) over the <br /> discretized time-interval with resolution N|
| DS | class-object | Object containing the method _vorticity_tensor, <br /> which computes the vorticity at   <br /> point x at time t|
| time | array (N) | time-interval [t_0, t_N] with resolution N|
| LAVD | array (Ny, Nx) | $ \mathrm{LAVD}_{t_0}^{t_N} $|

In [1]:
import sys, os

# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path
sys.path.append(parent_directory+"/General_Functions")

/Users/Alex_encinas_11_/OneDrive/TBarrier/TBarrier/2D/Gridded/Demos/LAVD


In [2]:
import numpy as np
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

def _LAVD(trajectory_grid, DS, time):
    
    lenT = time[-1]-time[0]
    
    X_grid, Y_grid = trajectory_grid[:,:,0,0], trajectory_grid[:,:,1,0]
    
    def _spatially_averaged_vorticity(trajectory_grid, DS, t):
        
        print(t)
            
        X_grid, Y_grid = trajectory_grid[:,:,0,0], trajectory_grid[:,:,1,0]
            
        omega = np.zeros(X_grid.shape)
            
        for i in range(omega.shape[0]):
            
            for j in range(omega.shape[1]):
            
                x = np.array([X_grid[i, j], Y_grid[i, j]])
    
                W = DS._vorticity_tensor(x, t)
                
                omega[i, j] = W[0, 1] - W[1, 0]
          
        return np.nanmean(omega.ravel())
                              
    spatially_averaged_vorticity = np.array(Parallel(n_jobs=DS.Ncores, verbose = 0)(delayed(_spatially_averaged_vorticity)(trajectory_grid, DS, t) for t in time))
    
    def parallelization(spatially_averaged_vorticity, k, t): 
                
        LVD = np.zeros((trajectory_grid.shape[0], trajectory_grid.shape[1]))
            
        for i in range(LVD.shape[0]):
            
            for j in range(LVD.shape[1]):
                    
                x = np.array([trajectory_grid[i, j, 0, k], trajectory_grid[i, j, 1, k]])

                if np.isfinite(x[0]) and np.isfinite(x[1]):
                    
                    W = DS._vorticity_tensor(x, t)
                    
                    omega = W[0, 1]-W[1, 0]
                
                    LVD[i, j] = np.abs(omega-spatially_averaged_vorticity)
                    
                else:
                        
                    LVD[i, j] = np.nan
                    
        return LVD
        
    LVD = np.array(Parallel(n_jobs=DS.Ncores, verbose = 0)(delayed(parallelization)(spatially_averaged_vorticity[k], k, t) for k, t in tqdm(enumerate(DS.time), total=len(DS.time))))
        
    import warnings
        
    # Suppress RunTimeWarning generated computing mean of empty slice
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        
        LAVD = np.nanmean(LVD, axis = 0)
        
    return LAVD/lenT